##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Github issues analyzer

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/GitHub_issues_analyzer.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook demonstrates how to use the Gemini API with function calling to interact with the GitHub API and display information about issues in a repository.

### What is PyGithub?
PyGitHub is a Python library to access the GitHub REST API. This library enables you to manage GitHub resources such as repositories, user profiles, and organizations in your Python applications.

### Install dependencies

In [1]:
%pip install -qU "google-genai>=1.0.0" "PyGithub"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Setup up yout API key
To run the following cell, your API key must be stored in a Colab Secret named **GOOGLE_API_KEY**. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the **Authentication** quickstart for an example.

You will also need a GitHub Personal Access Token stored as a Colab Secret named **GITHUB_API_KEY**. 

To get the **GITHUB_API_KEY** follow these instructions:

1. Navigate to your GitHub account settings, select `Developer settings`
2. then choose `Personal access tokens` and click on `Tokens (classic)` 
3. From there, you can generate a new token with the specific scopes (permissions) required for your application

check [GitHub documentation](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens) for more information
 

In [ ]:
from google import genai
from google.colab import userdata


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GITHUB_API_KEY = userdata.get("GITHUB_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

### Choose a model

In [3]:
MODEL_ID="gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

### Define GitHub API functions as tools
Now, define the Python functions that will be used as tools by the Gemini model. These functions will interact with the GitHub API using the PyGithub library. The docstrings are crucial as they provide the model with the necessary information to understand what each function does and what arguments it expects.


In [ ]:
from github import Github

g = Github(GITHUB_API_KEY)


def get_github_issues(repository_name: str):
    """
    Gets the open issues for a given GitHub repository.

    Args:
        repository_name: The name of the repository in the format 'owner/repo'.
    
    Returns:
        A list of dictionaries, where each dictionary represents an issue and contains its title, number, and URL.
    """
    try:
        repo = g.get_repo(repository_name)
        issues = repo.get_issues(state='open')
        
        issue_list = []
        for issue in issues:
            issue_list.append({
                'title': issue.title,
                'number': issue.number,
                'url': issue.html_url
            })
        return issue_list

    except Exception as e:
        return {"error": f"An error occurred: {e}"}

### Interact with the model
Now it's time to interact with the Gemini model. We'll create a ChatSession and pass the get_github_issues function as a tool. The model will automatically call the function with the correct arguments based on the user's prompt.


In [8]:
chat = client.chats.create(
    model=MODEL_ID,
    config={
        "tools": [get_github_issues]
    }
)

response = chat.send_message("Show me the open issues in the google-gemini/cookbook.")

print(response.text)

Here are the open issues in the google-gemini/cookbook repository:

*   **922**: Legal Retrieval Pipeline Cookbook with Gemini Embeddings using MLR & … (Pull Request) - https://github.com/google-gemini/cookbook/pull/922
*   **915**: added example of github actions (Pull Request) - https://github.com/google-gemini/cookbook/pull/915
*   **912**: Added batching_and_chunking.ipynb notebook (Pull Request) - https://github.com/google-gemini/cookbook/pull/912
*   **911**: AI-powered GitHub Action - https://github.com/google-gemini/cookbook/issues/911
*   **908**: quickstarts/Get_started_LiveAPI.py script doesn't work if using vertexai=True - https://github.com/google-gemini/cookbook/issues/908
*   **907**: Gemini 2.5 Flash Model Inconsistency Between API and Online Interface (Spatial Understanding) - https://github.com/google-gemini/cookbook/issues/907
*   **906**: Gemini Live API - Websocket Tool Call Response - BidiGenerateContentToolResponse - https://github.com/google-gemini/cookbook/issu

You can ask the model for other types of analysis such as what are the best issues to start from as a beginner,what are the most pressing issues I should look at. 

### Next Steps

- Learn more about PyGithub from the [PyGithub Docs](https://pygithub.readthedocs.io/en/stable/).
- You  some of your own tools the GitHub API has a vast range of functionalities such a as create_github_issue(),List_pll_requests(),get_issues_comments. 